In [19]:
import csv
import datetime
import os
import re
import pandas as pd

In [20]:
home_cities = ["bom","del","blr"]
college_city = "knu"
data_dir = "travel_day_data"

In [21]:
def get_files(file_re):
    files = []
    for filename in os.listdir(data_dir):
        if re.search(file_re, filename):
            files.append(data_dir + "/" + filename)
    return files

def get_csv_data(filename):
    data = []
    with open(filename, 'r') as f:
        b = csv.reader(f)
        for row in b:
            data.append(row[0])
    return data

In [22]:
def get_booking_date(filename):
        day = filename[-6:-4]
        month = filename[-9:-7]
        year = filename[-12:-10]
        return '20' + year + '-' + month + '-' + day

def get_row_info(row):
    
    
    def get_travel_year(row_date):
        if row_date.month < 9:
            return "2025"
        else:
            return "2024"
        
    def get_travel_month(month):
        if month < 10:
            return '0' + str(month)
        else:
            return str(month)
        
    def get_travel_day(day):
        if day < 10:
            return '0' + str(day)
        else:
            return str(day)
        
    if "There are no flights on this date" in row:
        price = -1
    else:
        price = int(row.split('â‚¹')[1].replace(',',''))
    x = datetime.datetime.strptime(row.split('\n')[1], "%a, %b %d")
    row_date = str(get_travel_year(x)) + "-" + get_travel_month(x.month) + "-" + get_travel_day(x.day)
    
    return [row_date, price]

In [23]:
all_data = []

for home_city in home_cities:
    
    file_re = home_city + "_to_" + college_city + ".*"
    files = get_files(file_re)
    for file in files:
        booking_date = get_booking_date(file)
        data = get_csv_data(file)
        for row in data:
            if len(row)!=0 and "Getting price" not in row:
                info = get_row_info(row)
                all_data.append([home_city, college_city, booking_date] + info)

    file_re = college_city + "_to_" + home_city + ".*"
    files = get_files(file_re)
    for file in files:
        booking_date = get_booking_date(file)
        data = get_csv_data(file)
        for row in data:
            if len(row)!=0 and "Getting price" not in row:
                info = get_row_info(row)
                all_data.append([college_city, home_city, booking_date] + info)

In [24]:
headings = ["source","destination","booking_date","travel_date","price"]
df = pd.DataFrame(all_data, columns=headings)
df.to_csv("complied_travel_data.csv",index=False)